Этот алгоритм сначала сортирует все изображения в указанной директории по имени. Затем он проходит по списку и извлекает каждые shift-число кадров через один, перемещая их в папку "deleted" из директории с изображениями.

Это нужно сделать в силу лимитов пространтсва Google Диска и для повышения уникальности кадров засчет удаления ближайших соседей. Так же способствует увеличению скорости обучения.

In [10]:
import os
import os
import shutil

def size_reduction(sample='train', shift=8):
    
    dlt_label = f'deleted_{sample}'
    path = f'safety_helmet_object_detection/{sample}/'
    
    # путь к новой папке в директории deleted
    unique_path = lambda folder_type: path + f'{dlt_label}/{folder_type}'

    # список всех снимков в выборке
    image_files = os.listdir(path + "images")
    image_files.sort()
    image_files = [x.split('.')[0] for x in image_files]

    # Создаем директорию deleted
    mk_folder = lambda folder: (os.makedirs(path+folder) 
                if not os.path.exists(path+folder) else None)
    mk_folder(dlt_label)
    mk_folder(f'{dlt_label}/images')
    mk_folder(f'{dlt_label}/labels')

    #список индексов на извлечение 
    indxs = list()
    shift += 1
    for i in range(0,len(image_files),shift):
        for j in range(1,shift): indxs.append(i+j)

    # список индексов на извлечение - каждый второй и третий кадры через один по всему списку image_files
    image_files = [j for i,j in enumerate(image_files) if i in indxs]
    
    # функция извлечения кадра или аннотации
    def move(src, folder_type='images', file_type='jpg'):
        # Путь к исходному файлу
        src = path + f'{folder_type}/{src}.{file_type}' 
        # Полный путь к целевому файлу
        dst = os.path.join(unique_path(folder_type), os.path.basename(src))
        # Перемещаем файл
        shutil.move(src, dst)

    # Цикл извлечения кадров
    for i in image_files:
        move(i, folder_type='images', file_type='jpg')
        move(i, folder_type='labels', file_type='txt')


In [ ]:
%%time
shift = 6
for sample in ['train', 'valid', 'test']:
    size_reduction(sample, shift)
